In [9]:
import pandas as pd
from sklearn.cluster import KMeans

In [16]:
# dataset preparation
url = "Median Household Income By Zip Code in Los Angeles County, California.html"
dfs = pd.read_html(url)
df_census = pd.read_csv("2010-census-populations-by-zip-code.csv")
geodata = pd.read_csv("us-zip-code-latitude-and-longitude.csv", sep=';')
df_income = dfs[0]
merged_data = df_census.join(df_income.set_index('Zip Code'), on='Zip Code')
dataset = merged_data[merged_data['Estimated Median Income'].notna()]
dataset_geo = dataset.join(geodata.set_index('Zip'), on='Zip Code', how='left')
dataset_geo.drop(columns=[
  'State', 'Timezone', 'Daylight savings time flag', 'geopoint'],
  inplace = True)
prepared_ds = dataset_geo[[
  "Zip Code", "City", "Community", "Estimated Median Income",
  "Longitude", "Latitude", "Total Population", "Median Age",
  "Total Males", "Total Females", "Total Households",
  "Average Household Size"]]
ppds = prepared_ds.drop(prepared_ds[prepared_ds['Estimated Median Income'] ==
                             '---'].index)
final_ds = ppds.dropna(axis=0)
final_ds['Estimated Median Income'] = \
    final_ds['Estimated Median Income'].map(lambda x: x.lstrip('$'))
final_ds['Estimated Median Income'] =\
    final_ds['Estimated Median Income'].str.replace(',','')
final_ds["Estimated Median Income"] =\
    pd.to_numeric(final_ds["Estimated Median Income"])
# final_ds['Residents'] = final_ds['Total Households'] * \
#                        final_ds['Average Household Size']

<ipython-input-16-be3c16717e67>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ds['Estimated Median Income'] = \
<ipython-input-16-be3c16717e67>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ds['Estimated Median Income'] =\
<ipython-input-16-be3c16717e67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [17]:
final_ds.shape

(279, 12)

In [18]:
grouped_clustering = final_ds.drop(['Zip Code', 'City', 'Community', 'Longitude', 'Latitude', 'Total Males',
                                    'Total Females'], 1)
#grouped_clustering['Residents'] = grouped_clustering['Total Households'] * \
#                                  grouped_clustering['Average Household Size']
grouped_clustering.head()

,Estimated Median Income,Total Population,Median Age,Total Households,Average Household Size
1,43360,57110,26.6,12971,4.40
2,37285,51223,25.5,11731,4.36
3,40598,66266,26.3,15642,4.22
4,49675,62180,34.8,22547,2.73
5,38491,37681,33.9,15044,2.50


In [19]:
kclusters = 7
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)
kmeans.labels_

array([3, 3, 3, 3, 3, 3, 3, 3, 6, 4, 6, 6, 6, 6, 3, 6, 3, 4, 3, 6, 4, 3,
       0, 0, 4, 0, 6, 3, 3, 3, 3, 0, 2, 0, 3, 6, 2, 6, 2, 0, 3, 4, 5, 0,
       3, 5, 5, 2, 3, 6, 3, 6, 6, 3, 5, 0, 0, 5, 2, 2, 6, 1, 1, 4, 1, 2,
       2, 3, 3, 6, 2, 2, 2, 0, 0, 5, 3, 6, 6, 4, 1, 4, 2, 4, 1, 1, 6, 1,
       1, 1, 5, 5, 4, 1, 5, 5, 5, 3, 6, 6, 6, 2, 2, 1, 5, 2, 5, 0, 2, 0,
       2, 5, 2, 6, 5, 0, 0, 2, 0, 0, 3, 4, 0, 2, 2, 5, 2, 4, 6, 5, 5, 2,
       6, 2, 3, 3, 5, 2, 3, 0, 2, 2, 3, 2, 3, 4, 3, 2, 5, 3, 3, 2, 5, 5,
       5, 2, 2, 1, 0, 6, 5, 5, 2, 2, 2, 2, 2, 1, 2, 5, 1, 6, 2, 6, 6, 3,
       2, 2, 5, 1, 5, 5, 1, 6, 0, 0, 5, 2, 2, 2, 2, 2, 5, 4, 4, 3, 4, 0,
       0, 2, 5, 2, 3, 5, 5, 2, 5, 5, 5, 5, 1, 5, 5, 5, 3, 4, 5, 3, 3, 6,
       2, 1, 2, 6, 2, 2, 2, 3, 2, 5, 3, 3, 2, 0, 4, 5, 2, 6, 2, 6, 3, 3,
       2, 5, 4, 0, 2, 0, 2, 2, 6, 6, 5, 4, 3, 3, 3, 2, 2, 3, 2, 5, 0, 2,
       2, 3, 2, 5, 6, 3, 4, 0, 6, 6, 4, 0, 0, 6, 6], dtype=int32)

In [20]:
final_ds.insert(0, 'Cluster Label', kmeans.labels_)
final_ds

,Cluster Label,Zip Code,City,Community,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size
1,3,90001,Los Angeles,"Los Angeles (South Los Angeles), Florence-Graham",43360,-118.24878,33.972914,57110,26.6,28468,28642,12971,4.40
2,3,90002,Los Angeles,"Los Angeles (Southeast Los Angeles, Watts)",37285,-118.24845,33.948315,51223,25.5,24876,26347,11731,4.36
3,3,90003,Los Angeles,"Los Angeles (South Los Angeles, Southeast Los ...",40598,-118.27600,33.962714,66266,26.3,32631,33635,15642,4.22
4,3,90004,Los Angeles,"Los Angeles (Hancock Park, Rampart Village, Vi...",49675,-118.30755,34.077110,62180,34.8,31302,30878,22547,2.73
5,3,90005,Los Angeles,"Los Angeles (Hancock Park, Koreatown, Wilshire...",38491,-118.30848,34.058911,37681,33.9,19299,18382,15044,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,4,93550,Palmdale,"Lake Los Angeles, Palmdale",45918,-118.08549,34.565480,74929,27.5,36414,38515,20864,3.58
313,0,93551,Palmdale,"City Ranch, Leona Valley, Palmdale",93055,-118.18495,34.606146,50798,37.0,25056,25742,15963,3.18
314,0,93552,Palmdale,Palmdale,65728,-118.03750,34.561207,38158,28.4,18711,19447,9690,3.93
315,6,93553,Pearblossom,"Juniper Hills, Pearblossom",51630,-117.89885,34.487552,2138,43.3,1121,1017,816,2.62


In [21]:
mean_df = final_ds.groupby('Cluster Label').mean()
mean_df.head(10)

,Zip Code,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size
Cluster Label,,,,,,,,,,
0,90957.161290,76916.354839,-118.221785,34.092376,49780.548387,35.077419,24596.451613,25184.096774,16902.580645,2.994839
1,90624.000000,155063.444444,-118.425995,34.024770,17967.666667,43.838889,8777.833333,9189.833333,7020.722222,2.479444
2,90996.514706,79922.632353,-118.233939,34.058584,25033.294118,39.075000,12147.411765,12885.882353,9411.073529,2.674412
3,90715.538462,49366.115385,-118.240645,34.029760,47583.500000,32.100000,23514.942308,24068.557692,14384.500000,3.341346
4,90965.545455,55132.000000,-118.200403,34.086971,82832.863636,30.304545,40963.590909,41869.272727,21976.409091,3.788182
5,90980.604167,107616.541667,-118.329351,34.098008,28755.125000,41.210417,14037.354167,14717.770833,11164.645833,2.540417
6,90949.025000,50822.700000,-118.205699,34.101665,17608.250000,35.947500,8972.000000,8636.250000,5897.800000,2.797250


In [25]:
mean_df['Factor'] = (mean_df['Estimated Median Income'] * mean_df['Total Population']) / 100

In [26]:
mean_df.head(10)

,Zip Code,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,Factor
Cluster Label,,,,,,,,,,,
0,90957.161290,76916.354839,-118.221785,34.092376,49780.548387,35.077419,24596.451613,25184.096774,16902.580645,2.994839,3.828938e+07
1,90624.000000,155063.444444,-118.425995,34.024770,17967.666667,43.838889,8777.833333,9189.833333,7020.722222,2.479444,2.786128e+07
2,90996.514706,79922.632353,-118.233939,34.058584,25033.294118,39.075000,12147.411765,12885.882353,9411.073529,2.674412,2.000727e+07
3,90715.538462,49366.115385,-118.240645,34.029760,47583.500000,32.100000,23514.942308,24068.557692,14384.500000,3.341346,2.349013e+07
4,90965.545455,55132.000000,-118.200403,34.086971,82832.863636,30.304545,40963.590909,41869.272727,21976.409091,3.788182,4.566741e+07
5,90980.604167,107616.541667,-118.329351,34.098008,28755.125000,41.210417,14037.354167,14717.770833,11164.645833,2.540417,3.094527e+07
6,90949.025000,50822.700000,-118.205699,34.101665,17608.250000,35.947500,8972.000000,8636.250000,5897.800000,2.797250,8.948988e+06


In [29]:
final_ds.loc[final_ds['Cluster Label'] == 4, final_ds
    .columns[[1] + list(range(2, final_ds.shape[1]))]]

,Zip Code,City,Community,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size
10,90011,Los Angeles,Los Angeles (Southeast Los Angeles),40940,-118.25868,34.007063,103892,26.2,52794,51098,22168,4.67
18,90019,Los Angeles,"Los Angeles (Arlington Heights, Country Club P...",56389,-118.34015,34.048411,64458,35.8,31442,33016,23344,2.70
21,90022,Los Angeles,East Los Angeles,46051,-118.15581,34.023662,67179,29.8,33216,33963,17023,3.94
25,90026,Los Angeles,"Los Angeles (Echo Park, Silver Lake)",65269,-118.26596,34.078510,67869,34.0,34515,33354,24956,2.68
42,90044,Los Angeles,"Athens, Los Angeles (South Los Angeles)",35981,-118.29158,33.953814,89779,28.6,43128,46651,25144,3.55
69,90201,Bell,"Bell, Bell Gardens, Cudahy",44046,-118.17385,33.972814,101279,27.8,50658,50621,24104,4.16
85,90250,Hawthorne,Hawthorne (Holly Park),56304,-118.35092,33.914614,93193,31.9,45113,48080,31087,2.98
87,90255,Huntington Park,"Huntington Park, Walnut Park",44962,-118.21773,33.978404,75066,29.1,37525,37541,18419,4.06
89,90262,Lynwood,Lynwood,52313,-118.20186,33.924642,69745,27.8,33919,35826,14669,4.57
99,90280,South Gate,South Gate,52321,-118.19490,33.944264,94396,29.4,46321,48075,23278,4.05


In [30]:
from geopy.geocoders import Nominatim
address = 'Los Angeles County, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [33]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np

# create map
map_clusters = folium.Map(location=[
    latitude, longitude], zoom_start=11)

cluster4 = final_ds.loc[final_ds['Cluster Label'] == 4]

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluster4['Latitude'],
                                  cluster4['Longitude'],
                                  cluster4['Zip Code'],
                                  cluster4['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [ ]:
# test

In [34]:
cluster4['Factor'] = cluster4['Total Population'] * cluster4['Estimated Median Income']
cluster4

<ipython-input-34-68c3ef09f06c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster4['Factor'] = cluster4['Total Population'] * cluster4['Estimated Median Income']


,Cluster Label,Zip Code,City,Community,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,Factor
10,4,90011,Los Angeles,Los Angeles (Southeast Los Angeles),40940,-118.25868,34.007063,103892,26.2,52794,51098,22168,4.67,4253338480
18,4,90019,Los Angeles,"Los Angeles (Arlington Heights, Country Club P...",56389,-118.34015,34.048411,64458,35.8,31442,33016,23344,2.70,3634722162
21,4,90022,Los Angeles,East Los Angeles,46051,-118.15581,34.023662,67179,29.8,33216,33963,17023,3.94,3093660129
25,4,90026,Los Angeles,"Los Angeles (Echo Park, Silver Lake)",65269,-118.26596,34.078510,67869,34.0,34515,33354,24956,2.68,4429741761
42,4,90044,Los Angeles,"Athens, Los Angeles (South Los Angeles)",35981,-118.29158,33.953814,89779,28.6,43128,46651,25144,3.55,3230338199
69,4,90201,Bell,"Bell, Bell Gardens, Cudahy",44046,-118.17385,33.972814,101279,27.8,50658,50621,24104,4.16,4460934834
85,4,90250,Hawthorne,Hawthorne (Holly Park),56304,-118.35092,33.914614,93193,31.9,45113,48080,31087,2.98,5247138672
87,4,90255,Huntington Park,"Huntington Park, Walnut Park",44962,-118.21773,33.978404,75066,29.1,37525,37541,18419,4.06,3375117492
89,4,90262,Lynwood,Lynwood,52313,-118.20186,33.924642,69745,27.8,33919,35826,14669,4.57,3648570185
99,4,90280,South Gate,South Gate,52321,-118.19490,33.944264,94396,29.4,46321,48075,23278,4.05,4938893116


In [35]:
cluster4.to_csv(path_or_buf='cluster4.csv')